<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy
import en_core_web_md

In [0]:
# avoid decoding problems
df = pd.read_csv("train.csv")
 
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [22]:
df.head()

id  qid1  ...                                          question2 is_duplicate
0   0     1  ...  What is the step by step guide to invest in sh...            0
1   1     3  ...  What would happen if the Indian government sto...            0
2   2     5  ...  How can Internet speed be increased by hacking...            0
3   3     7  ...  Find the remainder when [math]23^{24}[/math] i...            0
4   4     9  ...            Which fish would survive in salt water?            0

[5 rows x 6 columns]

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

In [0]:
qPair_train, qPair_test = train_test_split(questions,test_size=0.30,random_state =1234) 

In [0]:
tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [31]:
# en_vectors_web_md, having 300 vectors.
nlp = spacy.load("en_core_web_md")
# nlp = spacy.load('en_core_web_sm')

vecs1 = []
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1)
    # 300 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    # print(doc1[0], doc1[0].vector.shape)
    for i,word1 in enumerate(doc1):
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1[i] += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)


100%|██████████| 404290/404290 [1:18:33<00:00, 85.77it/s]


In [0]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [0]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [0]:
# dataframe of nlp features
df1.head()

id  is_duplicate  ...  fuzz_partial_ratio  longest_substr_ratio
0   0             0  ...                 100              0.982759
1   1             0  ...                  75              0.596154
2   2             0  ...                  54              0.166667
3   3             0  ...                  40              0.039216
4   4             0  ...                  56              0.175000

[5 rows x 17 columns]

In [0]:
# data before preprocessing 
df2.head()

id  freq_qid1  freq_qid2  ...  word_share  freq_q1+q2  freq_q1-q2
0   0          1          1  ...    0.434783           2           0
1   1          4          1  ...    0.200000           5           3
2   2          1          1  ...    0.166667           2           0
3   3          1          1  ...    0.000000           2           0
4   4          3          1  ...    0.100000           4           2

[5 rows x 12 columns]

In [0]:
df3.head()

id  ...                                         q2_feats_m
0   0  ...  [11.636040467482347, -9.770243690564083, -2.42...
1   1  ...  [7.601154750585556, -2.2477694768458605, -5.17...
2   2  ...  [0.4482018134810708, -2.639052846214988, -10.7...
3   3  ...  [-0.579327384630839, -3.7459775706132254, -1.4...
4   4  ...  [12.02181550860405, -8.95174354314804, 2.69811...

[5 rows x 3 columns]

In [0]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

0          1         2   ...        93        94        95
0  14.075324  -9.645537 -4.587416  ...  0.276098 -8.758635  4.792305
1   9.608312  -7.600832 -7.447780  ...  4.785889 -1.024974 -0.700264
2   5.450527  -9.478967  0.037324  ...  0.117646 -6.688697  9.870747
3  -9.742456  -4.595935 -5.212554  ...  9.489238  4.927465 -1.918715
4  18.715253 -11.789500 -1.434143  ...  0.171182 -7.547669  6.986244

[5 rows x 96 columns]

In [0]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

0         1          2   ...        93        94        95
0  11.636040 -9.770244  -2.426637  ... -0.735645 -7.677888  4.434125
1   7.601155 -2.247769  -5.177962  ...  4.356004 -1.179725  0.396846
2   0.448202 -2.639053 -10.709892  ... -5.409244 -9.038666  8.666798
3  -0.579327 -3.745978  -1.445257  ... -0.292013 -3.842984  4.686855
4  12.021816 -8.951744   2.698110  ... -3.144955 -6.637333 -0.293987

[5 rows x 96 columns]

In [0]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 221


In [0]:
# storing the final features to csv file
save_path='/content/drive/My Drive/Colab Notebooks/AppliedAI/Quora_Assignment/'
if not os.path.isfile(save_path+'final_features_new.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv(save_path+'final_features_new.csv')